In [0]:
%run ./silver_layer

In [0]:
try:
    logger.info("extracting patient revenue and city info")
    patient_details=(f'''CREATE OR REPLACE TABLE etl_incremental.gold.revenue_city_disease AS
    SELECT
        city,
        disease,
        COUNT(DISTINCT patient_id) AS total_patients,
        SUM(bill_amount) AS total_revenue,
        AVG(bill_amount) AS avg_bill
    FROM etl_incremental.silver.patient_silver
    WHERE is_current = true
    GROUP BY city, disease''')
    patient_details_df=spark.sql(patient_details)
    logger.info("patient revenue and city info extracted")
except exception as e:
    logger.error(e)


In [0]:
try:
    logger.info("extracting patient aging info")
    patient_aging_data=('''CREATE OR REPLACE TABLE etl_incremental.gold.age_band_analysis AS
        SELECT
            CASE
                WHEN age < 18 THEN 'Child'
                WHEN age BETWEEN 18 AND 40 THEN 'Adult'
                WHEN age BETWEEN 41 AND 60 THEN 'Middle Age'
                ELSE 'Senior'
            END AS age_group,
            COUNT(*) AS patient_count,
            AVG(bill_amount) AS avg_bill
        FROM etl_incremental.silver.patient_silver
        WHERE is_current = true
        GROUP BY age_group;
        ''')
    patient_aging_df=spark.sql(patient_aging_data)
    logger.info("patient aging info extracted")
except Exception as e:
    logger.error("error while extracting patient aging info")


In [0]:
try:
    logger.info("extracting high cost patient info")
    high_cost_patients=('''CREATE OR REPLACE TABLE etl_incremental.gold.high_cost_patients AS
            SELECT *
            FROM (
                SELECT *,
                    RANK() OVER (PARTITION BY city ORDER BY bill_amount DESC) AS rnk
                FROM etl_incremental.silver.patient_silver
                WHERE is_current = true
            )
            WHERE rnk <= 5;
            '''
        )
    high_cost_patients_df=spark.sql(high_cost_patients)
    logger.info("high cost patient info extracted")
except Exception as e:
    logger.error("error while extracting high cost patient info")